In [11]:
!pip install transformers torch

In [3]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
text="Hello BERT!"
encoding_text=tokenizer.encode(text)
print(encoding_text)


[101, 7592, 14324, 999, 102]


In [8]:
from transformers import BertTokenizer,BertModel
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model=BertModel.from_pretrained("bert-base-uncased")
Sentences=[ "Hello BERT!", "Attention Masking is important" ]
encoding_input=tokenizer(Sentences,return_tensors="pt", padding=True,truncation=True )
print(encoding_input)

print("Attention Mask" , encoding_input)




{'input_ids': tensor([[  101,  7592, 14324,   999,   102,     0,     0],
        [  101,  3086,  7308,  2075,  2003,  2590,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}
Attention Mask {'input_ids': tensor([[  101,  7592, 14324,   999,   102,     0,     0],
        [  101,  3086,  7308,  2075,  2003,  2590,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}


In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pretrained GPT-2 model and tokenizer
model_name = "gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Input text
text = "Once upon a time"
inputs = tokenizer.encode(text, return_tensors="pt")

# Generate text
# Set max_length for how long you want the generated portion to be
output = model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a man who lived in a village called Krakow. He was an old man, and he had a daughter. She was beautiful, but she was very shy. One day, she came to him and said, "I want to go to the opera." He said to her, "'You are not going to be able to do it, because you are too young." She said: "Well, I am going." So he took her to a theater


In [13]:
!pip install torch

In [19]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
# Load pretrained DistilBERT model and tokenizer
model_name = "distilbert-base-cased-distilled-squad"
model = DistilBertForQuestionAnswering.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Context and Question
context = "OpenAI is an organization that studies artificial intelligence. They aim to ensure that AI benefits all of humanity."
question = "What does OpenAI study?"

# Tokenize input and get model predictions
inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
input_ids = inputs["input_ids"].tolist()[0]

# Get Answer
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits

answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Answer: {answer}")


Answer: artificial intelligence


Named Entity Recognition (NER) using BERT.

NER is the task of classifying named entities in text into predefined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, etc.

Named Entity Recognition using BERT:

In [21]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load pretrained BERT model and tokenizer for NER
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = BertForTokenClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Sample text
text = "Elon Musk is the CEO of SpaceX."

# Tokenize input
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
inputs = tokenizer.encode(text, return_tensors="pt")

# Predict token labels
outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

# Convert label IDs to label names
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organization right after another organization
    "I-ORG",   # Organization
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

new_tokens, new_labels = [], []
for token, label_id in zip(tokens, predictions[0].numpy()):
    label = label_list[label_id]
    new_tokens.append(token)
    new_labels.append(label)

# Display tokens with their predicted labels
for token, label in zip(new_tokens, new_labels):
    print(f"{token} - {label}")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[CLS] - O
El - I-PER
##on - I-PER
Mu - I-PER
##sk - I-PER
is - O
the - O
CEO - O
of - O
Space - I-ORG
##X - I-ORG
. - O
[SEP] - O
